In [17]:
import pandas as pd

data = pd.read_csv("/Users/julienberthomier/code/AmElmo/Main_Projects/Covid_tracking_dashboard/csv_files/covid_cases_deaths_who.csv")

In [26]:
data

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...
264013,2023-01-16,ZW,Zimbabwe,AFRO,0,259947,0,5635
264014,2023-01-17,ZW,Zimbabwe,AFRO,0,259947,0,5635
264015,2023-01-18,ZW,Zimbabwe,AFRO,0,259947,0,5635
264016,2023-01-19,ZW,Zimbabwe,AFRO,0,259947,0,5635


In [25]:
data['Country'].nunique()

237

## Main functions

We will list here the main functions we will package to generate our charts and figures for our dashboard. 

We will use `France` as an example here, which will become a parameter in our package.

### Number of new cases (last day)

In [33]:
data_country = data[data["Country"] == "France"]
new_cases_last = data_country["New_cases"].iloc[-1]
new_cases_last

3997

### Number of new deaths (last day)

In [37]:
data_country = data[data["Country"] == "France"]
new_deaths_last = data_country["New_deaths"].iloc[-1]
new_deaths_last

63

### Number of cummulative deaths (latest)

In [36]:
data_country = data[data["Country"] == "France"]
cumulative_deaths_latest = data_country["Cumulative_deaths"].iloc[-1]
cumulative_deaths_latest

160051

In [30]:
data_country

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
80208,2020-01-03,FR,France,EURO,0,0,0,0
80209,2020-01-04,FR,France,EURO,0,0,0,0
80210,2020-01-05,FR,France,EURO,0,0,0,0
80211,2020-01-06,FR,France,EURO,0,0,0,0
80212,2020-01-07,FR,France,EURO,0,0,0,0
...,...,...,...,...,...,...,...,...
81317,2023-01-16,FR,France,EURO,4166,38353104,9,159727
81318,2023-01-17,FR,France,EURO,1235,38354339,80,159807
81319,2023-01-18,FR,France,EURO,8381,38362720,99,159906
81320,2023-01-19,FR,France,EURO,5894,38368614,82,159988
